In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

imputedDataset = pd.read_stata("/Users/burke/Documents/microsim/microsim/data/fullyImputedDataset.dta")
imputedDataset.head()
noRecentUSInmigration = imputedDataset.loc[~imputedDataset.timeInUS.isin([1,2,3])]
earlyCrossSectionOfNonImmigrants = noRecentUSInmigration.loc[noRecentUSInmigration.year==2001]
# here we just need to throw out the very young people for comparisons
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.loc[earlyCrossSectionOfNonImmigrants.age > 18 + 2]
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.sample(n=len(earlyCrossSectionOfNonImmigrants), weights=earlyCrossSectionOfNonImmigrants.WTINT2YR, replace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# on UCOD 1 = MI, 2=stroke, 3 = other cause of death
earlyCrossSectionOfNonImmigrants.ucod.value_counts(normalize=True, sort=False)

0    0.876929
1    0.041024
2    0.005483
3    0.076564
Name: ucod, dtype: float64

In [ ]:
import os
os.chdir("/Users/burke/Documents/research/bpCog/mcm")
from mcm.population import NHANESDirectSamplePopulation
from mcm.outcome import OutcomeType

pop = NHANESDirectSamplePopulation(n=10000, year=2001)
pop.advance(10)

In [ ]:
popData = pd.DataFrame({"index" : pop._people.index, 
                    "baseAge" : [person._age[0] for person in pop._people],
                    "finalAge" : [person._age[-1] for person in pop._people],
                   'mi' : [person.has_mi_during_simulation() for person in pop._people], 
                   'stroke' : [person.has_stroke_during_simulation() for person in pop._people], 
                   'miAge' : [person._outcomes[OutcomeType.MI][0][0] if len(person._outcomes[OutcomeType.MI]) > 0 else -1 for person in pop._people],
                   'strokeAge' : [person._outcomes[OutcomeType.STROKE][0][0] if len(person._outcomes[OutcomeType.STROKE]) > 0 else -1 for person in pop._people],
                   'fatalStroke' : [person.has_fatal_stroke() for person in pop._people],
                   'fatalMI' : [person.has_fatal_mi() for person in pop._people],

                   'dead' : [person.is_dead()  for person in pop._people]})

popData['age10'] = pd.qcut(popData.baseAge, 10, labels=False)
popData['nonCVDeath'] = False
popData.loc[((popData.dead) & (~popData.fatalStroke) & (~popData.fatalMI)), 'nonCVDeath'] = True

popData['cvDeath'] = False
popData.loc[((popData.dead) & (popData.fatalStroke) | (popData.fatalMI)), 'cvDeath'] = True



In [ ]:
popData[['baseAge', 'stroke', 'mi', 'dead', 'age10', 'fatalStroke', 'fatalMI', 'nonCVDeath', 'cvDeath']].groupby('age10').mean()

In [ ]:
popData[['baseAge', 'stroke', 'mi', 'dead', 'age10', 'fatalStroke', 'fatalMI', 'nonCVDeath']].mean()